In [ ]:
import pickle
import numpy as np

with open('medical_imaging_training.pkl', 'rb') as f:
    arr_list = np.array(pickle.load(f))

with open('medical_imaging_classifications.pkl', 'rb') as f:
    labels_np = np.array(pickle.load(f))

with open('medical_imaging_testing.pkl', 'rb') as f:
    test_list = np.array(pickle.load(f))

with open("training.txt", "w") as file:
    file.write(np.array2string(arr_list, threshold = 10000000000000000))

In [2]:
print(arr_list.shape)
print(labels_np.shape)
print(test_list.shape)

(48692, 100, 100)
(48692, 25)
(97, 100, 100)


In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Reshape
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

split_idx = int(len(arr_list) * 0.8)
training_images = arr_list[:split_idx]
training_labels = labels_np[:split_idx]
validation_images = arr_list[split_idx:]
validation_labels = labels_np[split_idx:]

training_images = np.expand_dims(training_images, axis=-1)
validation_images = np.expand_dims(validation_images, axis=-1)

##datagen = ImageDataGenerator(
##    rotation_range=20,
##    width_shift_range=0.2,
##    height_shift_range=0.2,
##    shear_range=0.2,
##    zoom_range=0.2,
##    horizontal_flip=True,
##    fill_mode='nearest'
##)

print(f"Training image shape: {training_images.shape}")
print(f"Validation image shape: {validation_images.shape}")

2025-05-20 11:21:21.011821: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-20 11:21:21.056131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-20 11:21:21.056162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-20 11:21:21.057256: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-20 11:21:21.064292: I tensorflow/core/platform/cpu_feature_guar

Training image shape: (38953, 100, 100, 1)
Validation image shape: (9739, 100, 100, 1)


In [4]:
labels = [
    "spinal_canal_stenosis_l1_l2",
    "spinal_canal_stenosis_l2_l3",
    "spinal_canal_stenosis_l3_l4",
    "spinal_canal_stenosis_l4_l5",
    "spinal_canal_stenosis_l5_s1",
    "left_neural_foraminal_narrowing_l1_l2",
    "left_neural_foraminal_narrowing_l2_l3",
    "left_neural_foraminal_narrowing_l3_l4",
    "left_neural_foraminal_narrowing_l4_l5",
    "left_neural_foraminal_narrowing_l5_s1",
    "right_neural_foraminal_narrowing_l1_l2",
    "right_neural_foraminal_narrowing_l2_l3",
    "right_neural_foraminal_narrowing_l3_l4",
    "right_neural_foraminal_narrowing_l4_l5",
    "right_neural_foraminal_narrowing_l5_s1",
    "left_subarticular_stenosis_l1_l2",
    "left_subarticular_stenosis_l2_l3",
    "left_subarticular_stenosis_l3_l4",
    "left_subarticular_stenosis_l4_l5",
    "left_subarticular_stenosis_l5_s1",
    "right_subarticular_stenosis_l1_l2",
    "right_subarticular_stenosis_l2_l3",
    "right_subarticular_stenosis_l3_l4",
    "right_subarticular_stenosis_l4_l5",
    "right_subarticular_stenosis_l5_s1"
]

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

input_layer = Input(shape=(100,100,1))

x = Conv2D(16, (3,3), activation='relu')(input_layer)
x = MaxPooling2D((2,2))(x)

x = Conv2D(32, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)

outputs = [Dense(3, activation='softmax', name=i)(x) for i in labels]

model = Model(inputs=input_layer, outputs=outputs)

2025-05-20 11:21:22.954308: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [5]:
import pandas as pd

train_labels = {labels[i]: training_labels[:, i] for i in range(25)}
val_labels = {labels[i]: validation_labels[:, i] for i in range(25)}

losses = {name: 'sparse_categorical_crossentropy' for name in model.output_names}

model.compile(optimizer='adam', loss=losses, metrics=['accuracy'])

model.compile(optimizer='adam',
              loss=losses,
              metrics=['accuracy'])

model.fit(
    training_images,
    train_labels,
    epochs = 10,
    batch_size = 200,
    validation_data=(validation_images, val_labels)  
)

Epoch 1/10
195/195 [==============================] - 22s 82ms/step - loss: 24.9653 - spinal_canal_stenosis_l1_l2_loss: 0.4495 - spinal_canal_stenosis_l2_l3_loss: 0.6692 - spinal_canal_stenosis_l3_l4_loss: 0.9864 - spinal_canal_stenosis_l4_l5_loss: 0.9913 - spinal_canal_stenosis_l5_s1_loss: 0.4518 - left_neural_foraminal_narrowing_l1_l2_loss: 0.2953 - left_neural_foraminal_narrowing_l2_l3_loss: 0.8242 - left_neural_foraminal_narrowing_l3_l4_loss: 1.0441 - left_neural_foraminal_narrowing_l4_l5_loss: 1.3547 - left_neural_foraminal_narrowing_l5_s1_loss: 1.5540 - right_neural_foraminal_narrowing_l1_l2_loss: 0.4499 - right_neural_foraminal_narrowing_l2_l3_loss: 1.1500 - right_neural_foraminal_narrowing_l3_l4_loss: 0.9154 - right_neural_foraminal_narrowing_l4_l5_loss: 1.2329 - right_neural_foraminal_narrowing_l5_s1_loss: 1.2636 - left_subarticular_stenosis_l1_l2_loss: 0.4880 - left_subarticular_stenosis_l2_l3_loss: 1.1341 - left_subarticular_stenosis_l3_l4_loss: 1.1417 - left_subarticular_st

In [6]:
predictions = model.predict(test_list)

4/4 [==============================] - 1s 15ms/step


In [7]:
print(np.array(predictions).shape)
averaged_data = np.mean(np.array(predictions), axis=1) 
print(averaged_data.shape) 
print(averaged_data)

(25, 97, 3)
(25, 3)
[[0.9527317  0.03623134 0.01103695]
 [0.888196   0.08146258 0.0303415 ]
 [0.79951286 0.12992187 0.07056502]
 [0.71040523 0.1341465  0.15544823]
 [0.9519105  0.03419125 0.01389819]
 [0.96480757 0.03371675 0.00147583]
 [0.90478003 0.08884386 0.00637626]
 [0.7635564  0.21086483 0.02557887]
 [0.6019472  0.31284407 0.0852088 ]
 [0.62688494 0.2631061  0.11000893]
 [0.9564207  0.03621636 0.00736301]
 [0.90077907 0.09607657 0.00314439]
 [0.7451804  0.23423736 0.02058232]
 [0.5874601  0.33875126 0.07378855]
 [0.63822865 0.25341216 0.1083591 ]
 [0.94007397 0.04362794 0.0162982 ]
 [0.8171101  0.13781177 0.04507811]
 [0.65505415 0.2469052  0.09804066]
 [0.42379797 0.31383923 0.26236284]
 [0.6852772  0.2284191  0.0863036 ]
 [0.932794   0.05445866 0.01274741]
 [0.845888   0.11420401 0.03990816]
 [0.64559966 0.2452561  0.10914415]
 [0.4191903  0.31109953 0.26971018]
 [0.68454456 0.21166027 0.10379524]]


In [8]:
submissions = pd.DataFrame(averaged_data)

for i in range(len(labels)):
    labels[i] = '44036939_' + labels[i]
    
submissions[3] = submissions[2]
submissions[2] = submissions[1]
submissions[1] = submissions[0]

submissions[0] = labels

In [9]:
print(submissions)
submissions.to_csv('output.csv', index=False)

                                                  0         1         2  \
0              44036939_spinal_canal_stenosis_l1_l2  0.952732  0.036231   
1              44036939_spinal_canal_stenosis_l2_l3  0.888196  0.081463   
2              44036939_spinal_canal_stenosis_l3_l4  0.799513  0.129922   
3              44036939_spinal_canal_stenosis_l4_l5  0.710405  0.134146   
4              44036939_spinal_canal_stenosis_l5_s1  0.951910  0.034191   
5    44036939_left_neural_foraminal_narrowing_l1_l2  0.964808  0.033717   
6    44036939_left_neural_foraminal_narrowing_l2_l3  0.904780  0.088844   
7    44036939_left_neural_foraminal_narrowing_l3_l4  0.763556  0.210865   
8    44036939_left_neural_foraminal_narrowing_l4_l5  0.601947  0.312844   
9    44036939_left_neural_foraminal_narrowing_l5_s1  0.626885  0.263106   
10  44036939_right_neural_foraminal_narrowing_l1_l2  0.956421  0.036216   
11  44036939_right_neural_foraminal_narrowing_l2_l3  0.900779  0.096077   
12  44036939_right_neural